# Connect with Google Drive
For this to work you are required to have your own google drive containing the specified folders in the paths mentioned in this Notebook

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path = "/content/drive/MyDrive/CollabData/StreetSmarts/stone/"
folder_path_val =  "/content/drive/MyDrive/CollabData/StreetSmarts/pres_val/"

# Import required libraries

In [ ]:
import numpy as np
from itertools import cycle
import cv2

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

In [ ]:
import collections
import os
import tempfile

from matplotlib import gridspec
from matplotlib import pyplot as plt
from PIL import Image
import urllib
import matplotlib
matplotlib.use('Agg')

import tensorflow as tf

from google.colab import files

# Import DeepLab Segmentation Network

In [ ]:
DatasetInfo = collections.namedtuple(
    'DatasetInfo',
    'num_classes, label_divisor, thing_list, colormap, class_names')


def _cityscapes_label_colormap():
  """Creates a label colormap used in CITYSCAPES segmentation benchmark.

  See more about CITYSCAPES dataset at https://www.cityscapes-dataset.com/
  M. Cordts, et al. "The Cityscapes Dataset for Semantic Urban Scene Understanding." CVPR. 2016.

  Returns:
    A 2-D numpy array with each row being mapped RGB color (in uint8 range).
  """
  colormap = np.zeros((256, 3), dtype=np.uint8)
  colormap[0] = [128, 64, 128]
  colormap[1] = [244, 35, 232]
  colormap[2] = [70, 70, 70]
  colormap[3] = [102, 102, 156]
  colormap[4] = [190, 153, 153]
  colormap[5] = [153, 153, 153]
  colormap[6] = [250, 170, 30]
  colormap[7] = [220, 220, 0]
  colormap[8] = [107, 142, 35]
  colormap[9] = [152, 251, 152]
  colormap[10] = [70, 130, 180]
  colormap[11] = [220, 20, 60]
  colormap[12] = [255, 0, 0]
  colormap[13] = [0, 0, 142]
  colormap[14] = [0, 0, 70]
  colormap[15] = [0, 60, 100]
  colormap[16] = [0, 80, 100]
  colormap[17] = [0, 0, 230]
  colormap[18] = [119, 11, 32]
  return colormap


def _cityscapes_class_names():
  return ('road', 'sidewalk', 'building', 'wall', 'fence', 'pole',
          'traffic light', 'traffic sign', 'vegetation', 'terrain', 'sky',
          'person', 'rider', 'car', 'truck', 'bus', 'train', 'motorcycle',
          'bicycle')


def cityscapes_dataset_information():
  return DatasetInfo(
      num_classes=19,
      label_divisor=1000,
      thing_list=tuple(range(11, 19)),
      colormap=_cityscapes_label_colormap(),
      class_names=_cityscapes_class_names())


def perturb_color(color, noise, used_colors, max_trials=50, random_state=None):
  """Pertrubs the color with some noise.

  If `used_colors` is not None, we will return the color that has
  not appeared before in it.

  Args:
    color: A numpy array with three elements [R, G, B].
    noise: Integer, specifying the amount of perturbing noise (in uint8 range).
    used_colors: A set, used to keep track of used colors.
    max_trials: An integer, maximum trials to generate random color.
    random_state: An optional np.random.RandomState. If passed, will be used to
      generate random numbers.

  Returns:
    A perturbed color that has not appeared in used_colors.
  """
  if random_state is None:
    random_state = np.random

  for _ in range(max_trials):
    random_color = color + random_state.randint(
        low=-noise, high=noise + 1, size=3)
    random_color = np.clip(random_color, 0, 255)

    if tuple(random_color) not in used_colors:
      used_colors.add(tuple(random_color))
      return random_color

  print('Max trial reached and duplicate color will be used. Please consider '
        'increase noise in `perturb_color()`.')
  return random_color


def color_panoptic_map(panoptic_prediction, dataset_info, perturb_noise):
  """Helper method to colorize output panoptic map.

  Args:
    panoptic_prediction: A 2D numpy array, panoptic prediction from deeplab
      model.
    dataset_info: A DatasetInfo object, dataset associated to the model.
    perturb_noise: Integer, the amount of noise (in uint8 range) added to each
      instance of the same semantic class.

  Returns:
    colored_panoptic_map: A 3D numpy array with last dimension of 3, colored
      panoptic prediction map.
    used_colors: A dictionary mapping semantic_ids to a set of colors used
      in `colored_panoptic_map`.
  """
  if panoptic_prediction.ndim != 2:
    raise ValueError('Expect 2-D panoptic prediction. Got {}'.format(
        panoptic_prediction.shape))

  semantic_map = panoptic_prediction // dataset_info.label_divisor
  instance_map = panoptic_prediction % dataset_info.label_divisor
  height, width = panoptic_prediction.shape
  colored_panoptic_map = np.zeros((height, width, 3), dtype=np.uint8)

  used_colors = collections.defaultdict(set)
  # Use a fixed seed to reproduce the same visualization.
  random_state = np.random.RandomState(0)

  unique_semantic_ids = np.unique(semantic_map)
  for semantic_id in unique_semantic_ids:
    semantic_mask = semantic_map == semantic_id
    if semantic_id in dataset_info.thing_list:
      # For `thing` class, we will add a small amount of random noise to its
      # correspondingly predefined semantic segmentation colormap.
      unique_instance_ids = np.unique(instance_map[semantic_mask])
      for instance_id in unique_instance_ids:
        instance_mask = np.logical_and(semantic_mask,
                                       instance_map == instance_id)
        random_color = perturb_color(
            dataset_info.colormap[semantic_id],
            perturb_noise,
            used_colors[semantic_id],
            random_state=random_state)
        colored_panoptic_map[instance_mask] = random_color
    else:
      # For `stuff` class, we use the defined semantic color.
      colored_panoptic_map[semantic_mask] = dataset_info.colormap[semantic_id]
      used_colors[semantic_id].add(tuple(dataset_info.colormap[semantic_id]))
  return colored_panoptic_map, used_colors

def create_sidwalk_mask(panoptic_prediction, dataset_info):
  if panoptic_prediction.ndim != 2:
    raise ValueError('Expect 2-D panoptic prediction. Got {}'.format(
        panoptic_prediction.shape))

  semantic_map = panoptic_prediction // dataset_info.label_divisor
  instance_map = panoptic_prediction % dataset_info.label_divisor
  height, width = panoptic_prediction.shape
  colored_panoptic_map = np.zeros((height, width), dtype=np.uint8)

  used_colors = collections.defaultdict(set)
  # Use a fixed seed to reproduce the same visualization.
  random_state = np.random.RandomState(0)

  unique_semantic_ids = np.unique(semantic_map)
  for semantic_id in unique_semantic_ids:
    semantic_mask = semantic_map == semantic_id
    if semantic_id == 1:
      colored_panoptic_map[semantic_mask] = 255

  return colored_panoptic_map
def IoU_calculation(panoptic_prediction, actual_mask):
  
  inter = np.logical_and(actual_mask, panoptic_prediction)
  union = np.logical_or(actual_mask, panoptic_prediction)
  iou_return = np.sum(inter) / np.sum(union)
  #print("IoU of Image: " + str(iou_return))
  return iou_return
 


def vis_segmentation(image,panoptic_prediction,dataset_info,file_path,file_name,perturb_noise=60):
  """Visualizes input image, segmentation map and overlay view."""
  plt.figure(figsize=(30, 20))
  grid_spec = gridspec.GridSpec(2, 2)

  ax = plt.subplot(grid_spec[0])
  plt.imshow(image)
  plt.axis('off')
  ax.set_title('input image', fontsize=20)

  ax = plt.subplot(grid_spec[1])
  panoptic_map, used_colors = color_panoptic_map(panoptic_prediction,
                                                 dataset_info, perturb_noise)
  plt.imshow(panoptic_map)
  plt.axis('off')
  ax.set_title('panoptic map', fontsize=20)

  ax = plt.subplot(grid_spec[2])
  plt.imshow(image)
  plt.imshow(panoptic_map, alpha=0.7)
  plt.axis('off')
  ax.set_title('panoptic overlay', fontsize=20)

  ax = plt.subplot(grid_spec[3])
  max_num_instances = max(len(color) for color in used_colors.values())
  # RGBA image as legend.
  legend = np.zeros((len(used_colors), max_num_instances, 4), dtype=np.uint8)
  class_names = []
  for i, semantic_id in enumerate(sorted(used_colors)):
    legend[i, :len(used_colors[semantic_id]), :3] = np.array(
        list(used_colors[semantic_id]))
    legend[i, :len(used_colors[semantic_id]), 3] = 255
    if semantic_id < dataset_info.num_classes:
      class_names.append(dataset_info.class_names[semantic_id])
    else:
      class_names.append('ignore')

  plt.imshow(legend, interpolation='nearest')
  ax.yaxis.tick_left()
  plt.yticks(range(len(legend)), class_names, fontsize=15)
  plt.xticks([], [])
  ax.tick_params(width=0.0, grid_linewidth=0.0)
  plt.grid('off')
  os.chdir(file_path)
  plt.savefig(file_name, bbox_inches='tight')
  


In [ ]:
MODEL_NAME = 'wide_resnet41_os16_panoptic_deeplab_cityscapes_trainfine_saved_model'  # @param ['resnet50_os32_panoptic_deeplab_cityscapes_crowd_trainfine_saved_model', 'resnet50_beta_os32_panoptic_deeplab_cityscapes_trainfine_saved_model', 'wide_resnet41_os16_panoptic_deeplab_cityscapes_trainfine_saved_model', 'swidernet_sac_1_1_1_os16_panoptic_deeplab_cityscapes_trainfine_saved_model', 'swidernet_sac_1_1_3_os16_panoptic_deeplab_cityscapes_trainfine_saved_model', 'swidernet_sac_1_1_4.5_os16_panoptic_deeplab_cityscapes_trainfine_saved_model', 'axial_swidernet_1_1_1_os16_axial_deeplab_cityscapes_trainfine_saved_model', 'axial_swidernet_1_1_3_os16_axial_deeplab_cityscapes_trainfine_saved_model', 'axial_swidernet_1_1_4.5_os16_axial_deeplab_cityscapes_trainfine_saved_model', 'max_deeplab_s_backbone_os16_axial_deeplab_cityscapes_trainfine_saved_model', 'max_deeplab_l_backbone_os16_axial_deeplab_cityscapes_trainfine_saved_model']


_MODELS = ('resnet50_os32_panoptic_deeplab_cityscapes_crowd_trainfine_saved_model',
           'resnet50_beta_os32_panoptic_deeplab_cityscapes_trainfine_saved_model',
           'wide_resnet41_os16_panoptic_deeplab_cityscapes_trainfine_saved_model',
           'swidernet_sac_1_1_1_os16_panoptic_deeplab_cityscapes_trainfine_saved_model',
           'swidernet_sac_1_1_3_os16_panoptic_deeplab_cityscapes_trainfine_saved_model',
           'swidernet_sac_1_1_4.5_os16_panoptic_deeplab_cityscapes_trainfine_saved_model',
           'axial_swidernet_1_1_1_os16_axial_deeplab_cityscapes_trainfine_saved_model',
           'axial_swidernet_1_1_3_os16_axial_deeplab_cityscapes_trainfine_saved_model',
           'axial_swidernet_1_1_4.5_os16_axial_deeplab_cityscapes_trainfine_saved_model',
           'max_deeplab_s_backbone_os16_axial_deeplab_cityscapes_trainfine_saved_model',
           'max_deeplab_l_backbone_os16_axial_deeplab_cityscapes_trainfine_saved_model')
_DOWNLOAD_URL_PATTERN = 'https://storage.googleapis.com/gresearch/tf-deeplab/saved_model/%s.tar.gz'

_MODEL_NAME_TO_URL_AND_DATASET = {
    model: (_DOWNLOAD_URL_PATTERN % model, cityscapes_dataset_information())
    for model in _MODELS
}

MODEL_URL, DATASET_INFO = _MODEL_NAME_TO_URL_AND_DATASET[MODEL_NAME]


In [ ]:
model_dir = tempfile.mkdtemp()

download_path = os.path.join(model_dir, MODEL_NAME + '.gz')
urllib.request.urlretrieve(MODEL_URL, download_path)

!tar -xzvf {download_path} -C {model_dir}

LOADED_MODEL = tf.saved_model.load(os.path.join(model_dir, MODEL_NAME))

wide_resnet41_os16_panoptic_deeplab_cityscapes_trainfine_saved_model/
wide_resnet41_os16_panoptic_deeplab_cityscapes_trainfine_saved_model/assets/
wide_resnet41_os16_panoptic_deeplab_cityscapes_trainfine_saved_model/saved_model.pb
wide_resnet41_os16_panoptic_deeplab_cityscapes_trainfine_saved_model/variables/
wide_resnet41_os16_panoptic_deeplab_cityscapes_trainfine_saved_model/variables/variables.data-00000-of-00001
wide_resnet41_os16_panoptic_deeplab_cityscapes_trainfine_saved_model/variables/variables.index


In [ ]:
def downscale_image(image,downscale=0.9,gaussian=3, generate = False):
  o_dims = None
  if gaussian != 0:
    #image = cv2.GaussianBlur(image,(gaussian,gaussian),cv2.BORDER_DEFAULT)
    image = cv2.bilateralFilter(image,gaussian,45,45)
  if downscale != 1:
    o_h = image.shape[0]
    o_w = image.shape[1]
    o_dims = (o_w,o_h)
    height = int(image.shape[0] * downscale)
    width = int(image.shape[1] * downscale)
    dims = (width, height)
    image = cv2.resize(image, dims, interpolation = cv2.INTER_AREA)
  if downscale != 1 and generate == False:
    image = cv2.resize(image, o_dims, interpolation = cv2.INTER_AREA)
  return image, o_dims

In [ ]:
def segment_image_and_overlay(img,downscale = 0.90, bilat = 3):
  alt_image, o_dims = downscale_image(img,downscale,bilat,generate= False)
  print("Starting")
  output = LOADED_MODEL(tf.cast(alt_image, tf.uint8))
  print("Ending")
  img_pred = create_sidwalk_mask(output['panoptic_pred'][0],DATASET_INFO)
  img_pred = np.divide(img_pred,255)
  seg_arr = np.stack((img_pred,)*3, axis=-1)
  img_pred = np.multiply(img,seg_arr) 
  return img_pred

# Build and load model for Classification Network

In [ ]:
image_size = 550
classifier_model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(32, (3,3), activation='relu',padding='same', input_shape=(image_size, image_size, 3)),
tf.keras.layers.MaxPooling2D(),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2D(32, (3,3),padding='same', activation='relu'),
tf.keras.layers.Conv2D(32, (3,3),padding='same', activation='relu'),
tf.keras.layers.MaxPooling2D(),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2D(32, (3,3),padding='same', activation='relu'),
tf.keras.layers.MaxPooling2D(),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2D(64, (3,3),padding='same', activation='relu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2D(64, (3,3),padding='same', activation='relu'),
tf.keras.layers.MaxPooling2D(),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.Dense(1, activation='sigmoid')])

classifier_model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

classifier_model.load_weights(folder_path+"BalancedML_V2.h5")

# Import PSPNet

In [ ]:
psp_folder_path = "drive/MyDrive/CollabData/SoAccessible/SidewalkDB/"

In [ ]:
! pip install git+https://github.com/Erik-Iuhas/image-segmentation-keras

from keras_segmentation.models.model_utils import transfer_weights
from keras_segmentation.pretrained import pspnet_101_cityscapes
from keras_segmentation.models.pspnet import pspnet_50,pspnet_101
from imgaug import augmenters as iaa
from keras_segmentation.models.unet import vgg_unet

load_model = pspnet_50( n_classes=2)
load_model.load_weights(psp_folder_path + "PSPNET_SIDEWALK.h5")

In [ ]:
def psp_net_classification(input_img):
  img = downscale_image(input_img)
  sqwared = cv2.resize(input_img,(473,473),interpolation = cv2.INTER_AREA)
  out = load_model.predict_segmentation(inp=sqwared)
  sq_out = np.stack((out,)*3, axis=-1)
  new_arr = np.array(np.multiply(sqwared,sq_out))
  mask_img = cv2.resize(new_arr.astype(np.uint8),(550,550),interpolation = cv2.INTER_AREA)
  x = np.expand_dims(mask_img, axis=0)
  images = np.vstack([x])
  result = classifier_model.predict(images, batch_size=10)
  if result[0]<0.5:
    return 1,mask_img
  else:
    return 0,mask_img

# Test models on example photos

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  
  img = np.array(cv2.imread(path))
  segmented_img = segment_image_and_overlay(img)
  mask_img = cv2.resize(segmented_img,(550,550),interpolation = cv2.INTER_AREA)
  x = np.expand_dims(mask_img, axis=0)
  plt.imshow(img/255.)
  plt.imshow(segmented_img/255.)
  images = np.vstack([x])
  result = classifier_model.predict(images, batch_size=10)
  print(result)
  if result[0]<0.5:
    print(fn + " contains sidewalk")
  else:
    print(fn + " no sidewalk")

# Set up Google Street View Code

In [ ]:
import math

def cord_compare(cord_1, cord_2):

  long_dif =  abs(cord_1.get("lng") - cord_2.get("lng"))*10000
  lat_dif = abs(cord_1.get("lat") - cord_2.get("lat"))*10000
  print("lat_dif:" + str(lat_dif) + ", long_dif" + str(long_dif))
  total_dif = math.sqrt(long_dif**2 + lat_dif**2)
  print(total_dif)
  return [long_dif,lat_dif,total_dif]

In [ ]:
pip install google-streetview

  Created wheel for google-streetview: filename=google_streetview-1.2.9-py3-none-any.whl size=9797 sha256=f9cc3ac6d232c8c949661f743b8287f511a8cbbc2767f95041361ee3abe6fe7d
  Stored in directory: /root/.cache/pip/wheels/3d/3c/22/66891d135d7de956a903e1eac58515b816e499df0a6f486c28
  Created wheel for kwconfig: filename=kwconfig-1.1.7-py3-none-any.whl size=4991 sha256=82d4298b5392c688db832ee48abf867211d443fc842a7d60595de9d9d8f0585a
  Stored in directory: /root/.cache/pip/wheels/75/99/be/6216a0ec525d7bf335628c0ac2463f9a24357c0f07870db419
Successfully built google-streetview kwconfig


In [ ]:
# Import google_streetview for the api module
import google_streetview.api
import requests
import json
import os
import math

# Classes=======================================================================


# Functions======================================================================
    
# cuts a line bewteen 2 points into small sections
def getMidPoint(latLong1, latLong2, num):
  points = []
  for x in range(num):
    la = (latLong1.get('lat') - latLong2.get('lat')) * ((x + 1) / (num + 1))
    lo = (latLong1.get("lng") - latLong2.get("lng")) * ((x + 1) / (num + 1))
    points.append([latLong1.get('lat') - la,latLong1.get("lng") - lo])
  return points

#Takes 2 points and gets images of the sides of the road at mids number of intervals
def getMidImages(point1, point2, mids, count,folder_name,fp):

    # Find the mid point
  midPoint = getMidPoint(point1,point2,mids)
    
  #find the angle of the road, TEMP FEATURE IN DEV
  angle = math.degrees(math.atan2((point2.get('lat')- point1.get('lat')),(point2.get('lng') - point1.get('lng'))))
  angle = 90 - angle

  #create the url
  url = "https://roads.googleapis.com/v1/snapToRoads?path="
  for x in range(mids):
      url = url + str(midPoint[x][0]) + "%2C" + str(midPoint[x][1]) + "%7C"
  
  url = url[:-len("%7C")] 
  print(url)
  url = url + "&key=(Add your own API Key)"
  
  
  #then snap the midpoint 
  payload={}
  headers = {}

  pointsJSON = requests.request("GET", url, headers=headers, data=payload)
  
  # get the point lat and long
  pointsJSONText = json.loads(pointsJSON.text)
  snapMidPoint = pointsJSONText["snappedPoints"]
  
  #go through each midPoint
  local = []
  for x in snapMidPoint:
      local.append(str(x["location"]["latitude"]) + ',' + str(x["location"]["longitude"]))
  #------------------------------------------------


  # Define parameters for each point with street view api
  for x in local:
    #LEFT IMAGE
    params = [{
        'size': '640x300', # max 640x640 pixels
        'location': x,
        'heading': str(angle - 90),
        'pitch': '-10',
        'fov': '110',
        'key': '(Add your own API Key)'
    }]
    # Create a results object
    #api_list = google_streetview.helpers.api_list()
    resultsLeft = google_streetview.api.results(params)
    # Download images to directory
    resultsLeft.download_links(r""+fp+folder_name)
    newName = fp+ folder_name + "/gsv_" + str(count) + "_left_" + x + ".jpg"
    os.rename(fp +folder_name +"/gsv_0.jpg", newName)
    
    #RIGHT IMAGE
    params[0]["heading"] = str(angle + 90)
    # Create a results object
    #api_list = google_streetview.helpers.api_list()
    resultsRight = google_streetview.api.results(params)
    # Download images to directory
    resultsRight.download_links(r""+ fp + folder_name)
    newName = fp+folder_name+"/gsv_" + str(count) + "_right_" + x + ".jpg"

    os.rename(fp+folder_name+"/gsv_0.jpg", newName)
    

# points.append(LatLong(45.411867488963466, -75.69636522942828))
# points.append(LatLong(45.412804807525546, -75.69413481826761))




In [ ]:
import json



def parse_data(cord_file):
  
  total_photos = 0
  road_map = []
  cords = json.load(cord_file)
  for i in range(0,len(cords)):
    if(i+1 >= len(cords)):
      continue
    cord_1, cord_2 = cords[i], cords[i+1]
    diff = cord_compare(cord_1,cord_2)[2]
    if diff == 0:
      pass
    elif diff <= 14:
      total_photos += math.ceil(diff)
      if math.ceil(diff)//2 == 0:
        snaps = 1
      else:
        snaps = math.ceil(math.ceil(diff)/2)
      road_map.append([cord_1,cord_2,snaps])
    elif diff > 14:
      total_photos+= 7*2
      road_map.append([cord_1,cord_2,7])
    print("TOTAL MAP PICTURES: " + str(total_photos))
  return road_map,total_photos



In [ ]:
import os 




def parse_data_grey(file_name):
  total_photos = 0
  road_map = []
  with open(file_name) as f:
    lines = f.readlines()
  for cord_str in lines:
    rm_latlng = cord_str.replace("LatLng(","").replace(")","")
    parse_str = rm_latlng.split(",")
    cord_1 = {"lat": float(parse_str[0]) , "lng" : float(parse_str[1])}
    cord_2 = {"lat": float(parse_str[2]) , "lng" : float(parse_str[3])}
    diff = cord_compare(cord_1,cord_2)[2]
    if diff == 0:
      pass
    elif diff <= 14:
      total_photos += math.ceil(diff)
      if math.ceil(diff)//2 == 0:
        snaps = 1
      else:
        snaps = math.ceil(math.ceil(diff)/2)
      road_map.append([cord_1,cord_2,snaps])
    elif diff > 14:
      total_photos+= 7*2
      road_map.append([cord_1,cord_2,7])
    print("TOTAL MAP PICTURES: " + str(total_photos))
  return road_map,total_photos


In [ ]:

total_photos = 0
for file_name in os.listdir(folder_path + "cords"):
  if not ".txt" in file_name:
      continue
  cord_file = open(folder_path + "cords/" + file_name)
  total_photos += parse_data(cord_file)[1]

print("Entire Map Photo count: " + str(total_photos))

lat_dif:1.7499999999870397, long_dif0.91999999995096
1.9770938267731264
TOTAL MAP PICTURES: 2
lat_dif:2.7899999999903002, long_dif1.5299999999740521
3.1819805153184824
TOTAL MAP PICTURES: 6
lat_dif:1.2600000000162481, long_dif0.6100000000230921
1.399892853067376
TOTAL MAP PICTURES: 8
lat_dif:2.2900000000447562, long_dif1.1399999999639476
2.5580656754905227
TOTAL MAP PICTURES: 11
lat_dif:1.8999999999635975, long_dif1.069999999998572
2.1805733190742784
TOTAL MAP PICTURES: 14
lat_dif:1.1899999999798183, long_dif0.6900000001053286
1.3755726080790214
TOTAL MAP PICTURES: 16
lat_dif:5.910000000000082, long_dif3.3499999999264674
6.79342329017619
TOTAL MAP PICTURES: 23
lat_dif:5.680000000012342, long_dif3.1300000000555883
6.485314179011545
TOTAL MAP PICTURES: 30
lat_dif:3.4000000000133923, long_dif2.0599999999149077
3.9753741961909057
TOTAL MAP PICTURES: 34
lat_dif:9.340000000008786, long_dif5.4200000001003446
10.798703625956767
TOTAL MAP PICTURES: 45
lat_dif:9.919999999965512, long_dif5.409999

# Download Gray

In [ ]:
file_name = "Batch7.txt"

file_str = file_name.replace(".txt","")
point = []
road_map = parse_data_grey(file_name)[0]
image_dict = {}
for x in range(len(road_map)):
    getMidImages(road_map[x][0], road_map[x][1], road_map[x][2], x, "stone_photos/" + file_str,folder_path)
    image_dict[x] = {"Cord1" : road_map[x][0] , "Cord2" : road_map[x][1], "ImageList" : [], "Right_List": [], "Left_List" : [], "dist": road_map[x][2]}
#Add downloaded images to image_dict
for fn in os.listdir(folder_path + "stone_photos/" + file_str):
  if not ".jpg" in fn:
    continue
  current_index = fn.split("_")[1]
  image_dict[int(current_index)]["ImageList"].append(fn)

with open(folder_path + "stone_photos/" + file_str + '/data.json', 'w') as f:
  json.dump(image_dict, f)

lat_dif:0.3399999999942338, long_dif1.6800000000216642
1.714059508905356
TOTAL MAP PICTURES: 2
lat_dif:0.42000000000541604, long_dif0.5299999999408556
0.6762396024648781
TOTAL MAP PICTURES: 3
lat_dif:0.3799999999642978, long_dif0.22999999998774
0.4441846462533647
TOTAL MAP PICTURES: 4
lat_dif:0.7999999999697138, long_dif4.880000000042628
4.945139027405356
TOTAL MAP PICTURES: 9
lat_dif:0.42000000000541604, long_dif0.45999999997548
0.6228964600814417
TOTAL MAP PICTURES: 10
lat_dif:0.0, long_dif0.0
0.0
TOTAL MAP PICTURES: 10
lat_dif:1.9100000000094042, long_dif1.069999999998572
2.189292123046367
TOTAL MAP PICTURES: 13
lat_dif:0.0, long_dif0.0
0.0
TOTAL MAP PICTURES: 13
lat_dif:0.0, long_dif0.0
0.0
TOTAL MAP PICTURES: 13
lat_dif:0.42000000000541604, long_dif0.06999999996537554
0.4257933771205255
TOTAL MAP PICTURES: 14
lat_dif:0.040000000041118255, long_dif0.0
0.040000000041118255
TOTAL MAP PICTURES: 15
lat_dif:0.040000000041118255, long_dif0.08000000008223651
0.0894427191919348
TOTAL MAP P

# Download Images from Google Street View

In [ ]:
limit = 0 
for file_name in os.listdir(folder_path + "cords"):

  fn_num = file_name.split("(")[1].split(")")[0]
  print(fn_num)
  queue_list =  range(53,54)
  if not int(fn_num) in queue_list:
    continue
  
  file_str = file_name.replace(".txt","")
  if not ".txt" in file_name:
      continue
  cord_file = open(folder_path + "cords/" + file_name)
  point = []
  road_map = parse_data(cord_file)[0]
  image_dict = {}
  for x in range(len(road_map)):
      getMidImages(road_map[x][0], road_map[x][1], road_map[x][2], x, "stone_photos/" + file_str)
      image_dict[x] = {"Cord1" : road_map[x][0] , "Cord2" : road_map[x][1], "ImageList" : [], "Right_List": [], "Left_List" : [], "dist": road_map[x][2]}
  #Add downloaded images to image_dict
  for fn in os.listdir(folder_path + "stone_photos/" + file_str):
    if not ".jpg" in fn:
      continue
    current_index = fn.split("_")[1]
    image_dict[int(current_index)]["ImageList"].append(fn)

  with open(folder_path + "stone_photos/" + file_str + '/data.json', 'w') as f:
    json.dump(image_dict, f)


16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
lat_dif:0.42000000000541604, long_dif1.1499999999387
1.2242957158560834
TOTAL MAP PICTURES: 2
lat_dif:0.0, long_dif0.0
0.0
TOTAL MAP PICTURES: 2
lat_dif:5.22999999994056, long_dif3.580000000056316
6.337925528103141
TOTAL MAP PICTURES: 9
https://roads.googleapis.com/v1/snapToRoads?path=45.28099999999999%2C-75.7173115
https://roads.googleapis.com/v1/snapToRoads?path=45.280916399999995%2C-75.7171824%7C45.280811799999995%2C-75.7171108%7C45.2807072%2C-75.71703919999999%7C45.2806026%2C-75.71696759999999


# Replace existing data with updated information.

In [ ]:
limit = 0 
for file_name in os.listdir(folder_path_val + "cords"):
  
  fn_num = file_name.split("(")[1].split(")")[0]
  print(fn_num)
  queue_list =  range(46,47)
  if not int(fn_num) in queue_list:
    continue
  
  file_str = file_name.replace(".txt","")

  existing_json = folder_path_val + "pres_val_photos/" + file_str + '/data.json'
  with open(existing_json) as json_file:
    old_dict = json.load(json_file)

  if not ".txt" in file_name:
    continue
  cord_file = open(folder_path_val + "cords/" + file_name)
  point = []
  road_map = parse_data(cord_file)[0]
  image_dict = {}
  for x in range(len(road_map)):
    image_dict[x] = {"Cord1" : road_map[x][0] , "Cord2" : road_map[x][1], "ImageList" : old_dict[str(x)].get("ImageList"), "Right_List": [], "Left_List" : [], "dist": road_map[x][2]}
  #Add downloaded images to image_dict
  for fn in os.listdir(folder_path_val + "pres_val_photos/" + file_str):
    if not ".jpg" in fn:
      continue
    current_index = fn.split("_")[1]
    image_dict[int(current_index)]["ImageList"].append(fn)

  with open(folder_path_val + "pres_val_photos/" + file_str + '/data.json', 'w') as f:
    json.dump(image_dict, f)

16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
lat_dif:1.0300000000285081, long_dif2.599999999972624
2.7965872058486525
TOTAL MAP PICTURES: 3
lat_dif:0.0, long_dif0.0
0.0
TOTAL MAP PICTURES: 3
lat_dif:1.980000000045834, long_dif1.4500000000339242
2.454159734059681
TOTAL MAP PICTURES: 6
lat_dif:0.0, long_dif0.0
0.0
TOTAL MAP PICTURES: 6
lat_dif:3.699999999966508, long_dif2.589999999997872
4.516425577792812
TOTAL MAP PICTURES: 11
lat_dif:4.849999999976262, long_dif3.659999999996444
6.076026662198226
TOTAL MAP PICTURES: 18
lat_dif:2.510000000057744, long_dif1.9899999999495321
3.2031546949982
TOTAL MAP PICTURES: 22
lat_dif:0.0, long_dif0.0
0.0
TOTAL MAP PICTURES: 22
lat_dif:2.6699999999379997, long_dif2.0600000000570162
3.37231374576919
TOTAL MAP PICTURES: 26
lat_dif:3.4000000000133923, long_dif2.589999999997872
4.274119792434466
TOTAL MAP PICTURES: 31
lat_dif:0.9899999999873899, long_dif0.6100000000230921
1.162841347735453
TOTAL MAP PICTURES: 

# Preform Classification Tests on Sidewalks

This section is responsible for testing accuracy and persision of the SmartML subsystem.


In [ ]:
import json
import re

folder_name = "pres_val_photos/"


true_values = {
               10 : {"left" : False, "right" : False},
               11 : {"left" : False, "right" : False}
              }

T_P = 0
F_P = 0
T_N = 0
F_N = 0
for fn in os.listdir(folder_path_val + folder_name):
  if "." in fn:
    continue
  with open(folder_path_val + folder_name + fn + '/data.json') as json_file:
      image_dict = json.load(json_file)
  
  fn_num = re.findall(r'\d+', fn)[0]
  #fn_num = fn.split("(")[1].split(")")[0]
  print(fn_num)
  queue_list =  range(10,12)
  if not int(fn_num) in queue_list:
    continue

  current_folder = folder_path_val + folder_name + fn + "/"
  try:
    os.mkdir(current_folder + "/seg_img/")
  except:
    print("exists")
  print(image_dict)
  
  for key in image_dict.keys():
    current_list =  image_dict[key].get("ImageList")

    for image_name in current_list:
      img = np.array(cv2.imread(current_folder + image_name))
      #psp_net_output, seg_psp = psp_net_classification(img)
      
      
      segmented_img = segment_image_and_overlay(img)
      #cv2.imwrite(current_folder + "/seg_img/" + image_name,seg_psp)
      
      mask_img = cv2.resize(segmented_img,(550,550),interpolation = cv2.INTER_AREA)
      x = np.expand_dims(mask_img, axis=0)
      
      plt.imshow(img/255.)
      plt.imshow(segmented_img/255.)
      
      images = np.vstack([x])
      result = classifier_model.predict(images, batch_size=1)

      r_result = np.round(result[0])
      if r_result == 0:
        o_result = 1
      else: 
        o_result = 0
      
      if "left" in image_name:
        #image_dict[key]["Left_List"].append(o_result)
        #image_dict[key]["Left_List"].append(psp_net_output)
        if(true_values[int(fn_num)]["left"]==o_result):
          if(true_values[int(fn_num)]["left"] == False):
            T_N+=1
          elif(true_values[int(fn_num)]["left"] == True):
            T_P+=1
        else:
          if(true_values[int(fn_num)]["left"] == False):
            F_P+=1
          elif(true_values[int(fn_num)]["left"] == True):
            F_N+=1

      elif "right" in image_name:
        #image_dict[key]["Right_List"].append(o_result)
        #image_dict[key]["Right_List"].append(psp_net_output)
        if(true_values[int(fn_num)]["right"]==o_result):
          if(true_values[int(fn_num)]["right"] == False):
            T_N+=1
          elif(true_values[int(fn_num)]["right"] == True):
            T_P+=1
        else:
          if(true_values[int(fn_num)]["right"] == False):
            F_P+=1
          elif(true_values[int(fn_num)]["right"] == True):
            F_N+=1
      
      
      #if psp_net_output ==1:
      #  print(image_name + " contains sidewalk")
      #else:
      #  print(image_name + " no sidewalk")

print("Output:")
print("True Positive: " + str(T_P))
print("True Negative: " + str(T_N))
print("False Negative: " + str(F_N))
print("False Positive: " + str(F_P))

1
3
4
5
6
7
8
9
10
exists
{'0': {'Cord1': {'lat': 45.37796, 'lng': -75.729347}, 'Cord2': {'lat': 45.377029, 'lng': -75.73146}, 'ImageList': ['gsv_0_right_45.37716209057933,-75.73121064170695.jpg', 'gsv_0_left_45.377854930953106,-75.72962111247334.jpg', 'gsv_0_right_45.377854930953106,-75.72962111247334.jpg', 'gsv_0_left_45.37773945952076,-75.72988603572558.jpg', 'gsv_0_right_45.37773945952076,-75.72988603572558.jpg', 'gsv_0_left_45.37762398730311,-75.73015095829251.jpg', 'gsv_0_right_45.37762398730311,-75.73015095829251.jpg', 'gsv_0_left_45.37750851430013,-75.73041588017414.jpg', 'gsv_0_right_45.37750851430013,-75.73041588017414.jpg', 'gsv_0_left_45.37739304051184,-75.7306808013704.jpg', 'gsv_0_right_45.37739304051184,-75.7306808013704.jpg', 'gsv_0_left_45.37727756593824,-75.73094572188135.jpg', 'gsv_0_right_45.37727756593824,-75.73094572188135.jpg', 'gsv_0_left_45.37716209057933,-75.73121064170695.jpg'], 'Right_List': [], 'Left_List': [], 'dist': 7}}
Starting
Ending
Starting
Ending
St

# Segment and Classify Images from Database

In [ ]:
import json
import re

folder_name = "stone_photos/"

for fn in os.listdir(folder_path + folder_name):
  if "." in fn:
    continue
  with open(folder_path + folder_name + fn + '/data.json') as json_file:
      image_dict = json.load(json_file)
  
  fn_num = re.findall(r'\d+', fn)[0]
  #fn_num = fn.split("(")[1].split(")")[0]
  print(fn_num)
  queue_list =  range(6,8)
  if not int(fn_num) in queue_list:
    continue

  current_folder = folder_path + folder_name + fn + "/"
  try:
    os.mkdir(current_folder + "/seg_img/")
  except:
    print("exists")
  print(image_dict)
  
  for key in image_dict.keys():
    current_list =  image_dict[key].get("ImageList")

    for image_name in current_list:
      img = np.array(cv2.imread(current_folder + image_name))
      psp_net_output, seg_psp = psp_net_classification(img)
      
      
      #segmented_img = segment_image_and_overlay(img)
      #cv2.imwrite(current_folder + "/seg_img/" + image_name,seg_psp)
      '''
      mask_img = cv2.resize(segmented_img,(550,550),interpolation = cv2.INTER_AREA)
      x = np.expand_dims(mask_img, axis=0)
      
      plt.imshow(img/255.)
      plt.imshow(segmented_img/255.)
      
      images = np.vstack([x])
      result = classifier_model.predict(images, batch_size=1)

      r_result = np.round(result[0])
      if r_result == 0:
        o_result = 1
      else: 
        o_result = 0
      '''
      if "left" in image_name:
        #image_dict[key]["Left_List"].append(o_result)
        image_dict[key]["Left_List"].append(psp_net_output)
      elif "right" in image_name:
        #image_dict[key]["Right_List"].append(o_result)
        image_dict[key]["Right_List"].append(psp_net_output)
      
      if psp_net_output ==1:
        print(image_name + " contains sidewalk")
      else:
        print(image_name + " no sidewalk")

  json_output = {}
  post_proc = {}
  for key in image_dict.keys():
    cur_entry = image_dict[key]
    left_mean = np.mean(cur_entry.get("Left_List"))
    
    right_mean = np.mean(cur_entry.get("Right_List"))
    

    side_val = 0
    if left_mean >= 0.40:
      left_sidewalk = "HAS SIDEWALK!"
      side_val += 1
    else:
      left_sidewalk = "no sidewalk!"

    if right_mean >= 0.40:
      right_sidewalk = "HAS SIDEWALK"
      side_val += 1
    else:
      right_sidewalk = "no sidewalk!"
    

    print("Segment " , key , " Left has ", left_sidewalk )
    print(cur_entry.get("Left_List"))
    print("Left Mean: " ,left_mean)
    print("Segment " , key , " Right has ", right_sidewalk)
    print(cur_entry.get("Right_List"))
    print("Right Mean: " ,right_mean)

    P1 = str(cur_entry.get("Cord1").get("lat")) +","+ str(cur_entry.get("Cord1").get("lng")) 
    P2 = str(cur_entry.get("Cord2").get("lat")) +","+ str(cur_entry.get("Cord2").get("lng")) 

    P1_P2 = "P1:" + P1 + "_P2:" + P2
    P2_P1 = "P1:" + P2 + "_P2:" + P1

    
    post_proc[key] = {"Cord1": {"lat": cur_entry.get("Cord1").get("lat"), "lng": cur_entry.get("Cord1").get("lng")},
                      "Cord2": {"lat": cur_entry.get("Cord2").get("lat"), "lng": cur_entry.get("Cord2").get("lng")},
                      "dist" : cur_entry.get("dist"), "L_mean": left_mean, "R_mean" : right_mean}

    json_output[P1_P2] = {"sidewalkStatus" : side_val}
    json_output[P2_P1] = {"sidewalkStatus" : side_val}
    
  with open(current_folder + 'sidewalk_data_PSP_101.json', 'w') as f:
    json.dump(json_output, f)
  with open(current_folder + 'sidewalk_post_PSP_101.json', 'w') as f:
    json.dump(post_proc, f)
  print(image_dict)
  print(json_output)

# Post Processing Classification

In this section I am looking to smooth out data and improve sidewalk classification depending on surrounding classifications made.

In [ ]:

correction_count = 0
MODEL_COUNT = 1
total_points = 0
for fn in os.listdir(folder_path + "stone_photos/"):
  if "(" not in fn:
    continue
  with open(folder_path + "stone_photos/" + fn + '/sidewalk_post_PSP_101_50.json') as json_file:
      image_dict = json.load(json_file)
  

  

  current_folder = folder_path + "stone_photos/" + fn + "/"
  fn_num = fn.split("(")[1].split(")")[0]
  print(fn_num)

  json_output = {}
  for key in image_dict.keys():
    int_key = int(key)
    prev_key = image_dict.get(str(int_key-1))
    post_key = image_dict.get(str(int_key+1))
    cur_key = image_dict.get(key)
    #Left Check
    THRESH = 0.4 
    if (prev_key != None and post_key != None):
      if(cur_key.get("dist") == 1 or (cur_key.get("dist") == 2 and cur_key.get("L_mean") == 0.5)):
        if((prev_key["L_mean"] < THRESH and post_key["L_mean"] < THRESH) or (prev_key["L_mean"] >= THRESH and post_key["L_mean"] >= THRESH)):
          cur_key["L_mean"] = (cur_key["L_mean"]*MODEL_COUNT*cur_key["dist"] + prev_key["L_mean"]*MODEL_COUNT*prev_key["dist"] + post_key["L_mean"]*MODEL_COUNT*post_key["dist"])/3
          correction_count+= 1
      if(cur_key.get("dist") == 1 or (cur_key.get("dist") == 2 and cur_key.get("R_mean") == 0.5)):
        if((prev_key["R_mean"] < THRESH and post_key["R_mean"] < THRESH) or (prev_key["R_mean"] >= THRESH and post_key["R_mean"] >= THRESH)):
          cur_key["R_mean"] = (cur_key["R_mean"]*MODEL_COUNT*cur_key["dist"] + prev_key["R_mean"]*MODEL_COUNT*prev_key["dist"] + post_key["R_mean"]*MODEL_COUNT*post_key["dist"])/3
          correction_count+= 1
    total_points+=2 
    side_val = 0
    if cur_key["L_mean"] >= THRESH :
      left_sidewalk = "HAS SIDEWALK!"
      side_val += 1
    else:
      left_sidewalk = "no sidewalk!"

    if cur_key["R_mean"] >= THRESH:
      right_sidewalk = "HAS SIDEWALK"
      side_val += 1
    else:
      right_sidewalk = "no sidewalk!"
    

    print("Segment " , key , " Left has ", left_sidewalk )
    print("Segment " , key , " Right has ", right_sidewalk)

    P1 = str(cur_key.get("Cord1").get("lat")) +","+ str(cur_key.get("Cord1").get("lng")) 
    P2 = str(cur_key.get("Cord2").get("lat")) +","+ str(cur_key.get("Cord2").get("lng")) 

    P1_P2 = "P1:" + P1 + "_P2:" + P2
    P2_P1 = "P1:" + P2 + "_P2:" + P1


    json_output[P1_P2] = {"sidewalkStatus" : side_val}
    json_output[P2_P1] = {"sidewalkStatus" : side_val}
    
  with open(current_folder + 'sidewalk_data_denoise_psp_101.json', 'w') as f:
    json.dump(json_output, f)

print("Total Count: " + str(total_points))
print("Correction Count: " + str(correction_count))

23
Segment  0  Left has  no sidewalk!
Segment  0  Right has  no sidewalk!
Segment  1  Left has  no sidewalk!
Segment  1  Right has  no sidewalk!
Segment  2  Left has  no sidewalk!
Segment  2  Right has  no sidewalk!
Segment  3  Left has  no sidewalk!
Segment  3  Right has  no sidewalk!
Segment  4  Left has  no sidewalk!
Segment  4  Right has  no sidewalk!
Segment  5  Left has  no sidewalk!
Segment  5  Right has  no sidewalk!
Segment  6  Left has  no sidewalk!
Segment  6  Right has  no sidewalk!
Segment  7  Left has  no sidewalk!
Segment  7  Right has  no sidewalk!
Segment  8  Left has  HAS SIDEWALK!
Segment  8  Right has  no sidewalk!
17
Segment  0  Left has  no sidewalk!
Segment  0  Right has  HAS SIDEWALK
Segment  1  Left has  no sidewalk!
Segment  1  Right has  HAS SIDEWALK
Segment  2  Left has  no sidewalk!
Segment  2  Right has  HAS SIDEWALK
Segment  3  Left has  no sidewalk!
Segment  3  Right has  HAS SIDEWALK
Segment  4  Left has  HAS SIDEWALK!
Segment  4  Right has  no sidewalk

# Merge database into one *file*

In [ ]:
import json
entire_dict = {}
for fn in os.listdir(folder_path + folder_name):
  if "." in fn:
    continue
  try:
    with open(folder_path + folder_name + fn + '/sidewalk_data_denoise_psp_101.json') as json_file:
        image_dict = json.load(json_file)
  except:
    with open(folder_path + folder_name + fn + '/sidewalk_data_PSP_101.json') as json_file:
        image_dict = json.load(json_file)

  entire_dict = {**entire_dict , **image_dict}


print(len(entire_dict))
with open(folder_path + folder_name + 'Final_101.json', 'w') as f:
  json.dump(entire_dict, f)


1918


# Extra

In [ ]:
json_output = {}
for key in image_dict.keys():
  cur_entry = image_dict[key]
  left_mean = np.mean(cur_entry.get("Left_List"))
  right_mean = np.mean(cur_entry.get("Right_List"))

  side_val = 0
  if left_mean < 0.65:
    left_sidewalk = "HAS SIDEWALK!"
    side_val += 1
  else:
    left_sidewalk = "no sidewalk!"

  if right_mean < 0.65:
    right_sidewalk = "HAS SIDEWALK"
    side_val += 1
  else:
    right_sidewalk = "no sidewalk!"
  

  print("Segment " , key , " Left has ", left_sidewalk )
  print(left_mean)
  print("Segment " , key , " Right has ", right_sidewalk)
  print(right_mean)

  

  json_output[key] = {"Cord1" : cur_entry.get("Cord1"), "Cord2" : cur_entry.get("Cord2"), "has_sidewalk" : side_val}
  json_output[key+"_f"] = {"Cord2" : cur_entry.get("Cord1"), "Cord1" : cur_entry.get("Cord2"), "has_sidewalk" : side_val}

with open(current_folder + 'sidewalk_data.json', 'w') as f:
  json.dump(json_output, f)


Segment  0  Left has  HAS SIDEWALK!
0.0
Segment  0  Right has  HAS SIDEWALK
0.0
Segment  1  Left has  no sidewalk!
0.6666667
Segment  1  Right has  HAS SIDEWALK
0.0
Segment  2  Left has  HAS SIDEWALK!
0.0
Segment  2  Right has  HAS SIDEWALK
0.0
Segment  3  Left has  HAS SIDEWALK!
0.0
Segment  3  Right has  HAS SIDEWALK
0.33333334
Segment  4  Left has  HAS SIDEWALK!
0.0
Segment  4  Right has  HAS SIDEWALK
0.33333334
Segment  5  Left has  no sidewalk!
1.0
Segment  5  Right has  no sidewalk!
0.6666667
Segment  6  Left has  HAS SIDEWALK!
0.33333334
Segment  6  Right has  no sidewalk!
0.6666667
Segment  7  Left has  no sidewalk!
0.6661658
Segment  7  Right has  HAS SIDEWALK
0.33333334
Segment  8  Left has  HAS SIDEWALK!
0.33333334
Segment  8  Right has  HAS SIDEWALK
0.0
